<a href="https://colab.research.google.com/github/garapatimurali/ColabNoteBooks/blob/main/VGG11_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class VGG11_NET(nn.Module):
    def __init__(self):
        super(VGG11_NET, self).__init__()
        self.conv1_1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)

        self.conv2_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)

        self.conv3_1 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)

        self.conv4_1 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

        self.conv5_1 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(in_features=25088, out_features=4096)
        self.fc2 = nn.Linear(in_features=4096, out_features=4096)
        self.fc3 = nn.Linear(in_features=4096, out_features=1000)

    def forward(self, x):
        x = F.relu(self.conv1_1(x))
        x = self.maxpool(x)

        x = F.relu(self.conv2_1(x))
        x = self.maxpool(x)

        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = self.maxpool(x)

        x = F.relu(self.conv4_1(x))
        x = F.relu(self.conv4_2(x))
        x = self.maxpool(x)

        x = F.relu(self.conv5_1(x))
        x = F.relu(self.conv5_2(x))
        x = self.maxpool(x)

        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.5) # dropout included to combat overfitting
        x = F.relu(self.fc2(x))
        x = F.dropout(x, 0.5)
        x = self.fc3(x)

        return x

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import time

from torch import nn
from torch.utils.data import DataLoader, random_split

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
def train(dataloader, model, loss_fn, optimizer, epoch, num_epochs):
    loss_var = 0
    for idx, (images, labels) in enumerate(dataloader):
        images = images.to(device=device)
        labels = labels.to(device=device)

        # Forward pass
        pred = model(images)
        loss = loss_fn(pred, labels)

        # Backward propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_var += loss.item()
        if idx%100 == 0:
          print(f'Epoch [{epoch+1}/{num_epochs}] || Step [{idx+1}/{len(dataloader)}] || Loss:{loss_var/len(dataloader)}')

    print(f'Loss at epoch {epoch+1} || {loss_var/len(dataloader)}')

def test(dataloader, model):
    with torch.no_grad():
        correct = 0
        samples = 0
        for idx, (images, labels) in enumerate(dataloader):
            images = images.to(device=device)
            labels = labels.to(device=device)

            outputs = model(images)
            _, preds = outputs.max(1)
            correct += (preds == labels).sum()
            samples += preds.size(0)
        
        print(f"accuracy {float(correct) / float(samples) * 100:.2f} percentage || Correct {correct} out of {samples} samples")


In [8]:
batch_size = 64
num_epochs = 5
lr = 1e-4
class_size = 14

train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(p=0.7),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# prep the train, validation and test data
torch.manual_seed(2022)
train_data = torchvision.datasets.CIFAR10("data/", train=True, download=True, transform=train_transform)
val_size = 10000
train_size = len(train_data) - val_size
train_data, val_data = random_split(train_data, [train_size, val_size])
test_data = torchvision.datasets.CIFAR10('data/', train=False, download=True, transform=test_transform)

# train, val and test datasets to the dataloader
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

model = VGG11_NET()
model = model.to(device=device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
for epoch in range(num_epochs):
    t0 = time.time()
    train(train_loader, model, loss_fn, optimizer, epoch, num_epochs)
    print('{} seconds'.format(time.time() - t0))
    test(val_loader, model)

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/5] || Step [1/625] || Loss:0.011060655975341796
Epoch [1/5] || Step [101/625] || Loss:0.4415367923736572
Epoch [1/5] || Step [201/625] || Loss:0.8191426429748535
Epoch [1/5] || Step [301/625] || Loss:1.1653001123428344
Epoch [1/5] || Step [401/625] || Loss:1.47173774433136
Epoch [1/5] || Step [501/625] || Loss:1.7484215721130372
Epoch [1/5] || Step [601/625] || Loss:2.010126449584961
Loss at epoch 1 || 2.070697117805481
380.8392357826233 seconds
accuracy 41.90 percentage || Correct 4190 out of 10000 samples
Epoch [2/5] || Step [1/625] || Loss:0.0026911062240600587
Epoch [2/5] || Step [101/625] || Loss:0.24705821743011475
Epoch [2/5] || Step [201/625] || Loss:0.476221409034729
Epoch [2/5] || Step [301/625] || Loss:0.6965483976364135
Epoch [2/5] || Step [401/625] || Loss:0.9054147047996521
Epoch [2/5] || Step [501/625] || Loss:1.1073892016410827
Epoch [2/5] || Step [601/625] || Loss:1.2969077447891235
Lo